In [32]:
import json
import requests
import pandas as pd
from pathlib import Path
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path
from sklearn import cluster, covariance, manifold
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import warnings
warnings.filterwarnings('ignore')


In [33]:
events=pd.read_csv(
    Path('Institutional_Adoption_Events_positive_in_2021.csv'),
    index_col='DATE',
    parse_dates=True,
    infer_datetime_format=True
)
events=events.fillna(0)
events.head()

,NEWS,CATEGORY,Institutional Adopt
DATE,,,
2021-01-05,JP Morgan: BTC Could Hit $146K,Banking,1
2021-01-28,Dorsey Positive BTC Tweet,Corporate,1
2021-01-28,Musk Positive BTC Tweet,Musk Tweet,1
2021-02-08,Tesla Accepts BTC as Payment,Corporate,1
2021-02-08,SEC Filing: Tesla Bought $1.5 Billion in BTC,Government,1


In [34]:
events=events.drop(['NEWS', 'CATEGORY'], axis=1)
events.head()

,Institutional Adopt
DATE,
2021-01-05,1
2021-01-28,1
2021-01-28,1
2021-02-08,1
2021-02-08,1


In [35]:
prices=pd.read_csv(
    Path('crypto_market_caps.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True

)
prices.head()

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
date,,,,,,,,,,,,,,,,,,,,,
2021-05-29,0.000000,-0.087474,-0.084921,-0.178955,-0.069173,-0.075526,-0.116392,-0.141745,-0.110146,-0.081747,...,-0.086776,-0.107659,-0.161449,-0.144139,-0.127478,-0.204751,-0.148758,-0.141004,-0.074100,-0.154125
2021-05-30,0.231838,-0.029180,-0.077416,-0.050995,-0.020782,-0.021363,-0.048031,-0.045486,-0.063362,-0.056490,...,-0.101236,-0.070963,-0.058184,-0.062235,-0.049803,-0.049462,-0.048317,-0.080900,-0.092101,-0.066768
2021-05-31,0.238859,0.067797,0.092618,-0.037861,0.022546,0.022048,0.035723,0.030508,0.015360,0.099605,...,0.095878,0.008402,0.111175,0.081288,0.070708,0.119675,0.043991,0.033885,0.081935,0.058975
2021-06-01,-0.059507,0.106957,0.026690,-0.004131,-0.004840,0.045575,0.130062,0.145793,0.143006,0.109167,...,0.102625,0.202528,0.106781,0.178813,0.082806,0.195291,0.104627,0.063740,0.081550,0.123031
2021-06-02,-0.205798,-0.007821,-0.032677,0.333030,-0.024114,-0.014893,-0.025359,-0.050999,-0.015254,0.000810,...,-0.030430,-0.194083,-0.023898,-0.023796,0.002862,-0.057441,-0.031295,0.050371,0.002730,-0.042151


In [36]:
combined=pd.merge(prices,events,left_index=True, right_index=True)
combined.head()

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida,Institutional Adopt
2021-01-05,0.0,0.0,-0.088201,-0.122700,0.0,-0.045453,0.037039,0.120509,-0.108976,0.046063,...,0.0,-0.098702,0.142612,-0.026812,-0.520328,0.059270,-0.080577,0.018532,0.080724,1
2021-01-28,0.0,0.0,-0.036659,0.178912,0.0,-0.065799,-0.087956,-0.084846,-0.098515,-0.098414,...,0.0,-0.056205,0.037974,-0.093725,-0.182521,-0.139449,-0.112352,-0.104540,-0.068550,1
2021-01-28,0.0,0.0,-0.036659,0.178912,0.0,-0.065799,-0.087956,-0.084846,-0.098515,-0.098414,...,0.0,-0.056205,0.037974,-0.093725,-0.182521,-0.139449,-0.112352,-0.104540,-0.068550,1
2021-02-08,0.0,0.0,0.071374,-0.045171,0.0,-0.012313,-0.041916,0.078294,-0.035286,0.053139,...,0.0,-0.054602,0.017914,-0.047580,-0.051430,-0.023283,0.174763,0.089021,-0.024576,1
2021-02-08,0.0,0.0,0.071374,-0.045171,0.0,-0.012313,-0.041916,0.078294,-0.035286,0.053139,...,0.0,-0.054602,0.017914,-0.047580,-0.051430,-0.023283,0.174763,0.089021,-0.024576,1


In [37]:
combined = combined.dropna(axis=0, how='all')
combined = combined.dropna(axis=1)
combined

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida,Institutional Adopt
2021-01-05,0.000000,0.000000,-0.088201,-0.122700,0.000000,-0.045453,0.037039,0.120509,-0.108976,0.046063,...,0.000000,-0.098702,0.142612,-0.026812,-0.520328,0.059270,-0.080577,0.018532,0.080724,1
2021-01-28,0.000000,0.000000,-0.036659,0.178912,0.000000,-0.065799,-0.087956,-0.084846,-0.098515,-0.098414,...,0.000000,-0.056205,0.037974,-0.093725,-0.182521,-0.139449,-0.112352,-0.104540,-0.068550,1
2021-01-28,0.000000,0.000000,-0.036659,0.178912,0.000000,-0.065799,-0.087956,-0.084846,-0.098515,-0.098414,...,0.000000,-0.056205,0.037974,-0.093725,-0.182521,-0.139449,-0.112352,-0.104540,-0.068550,1
2021-02-08,0.000000,0.000000,0.071374,-0.045171,0.000000,-0.012313,-0.041916,0.078294,-0.035286,0.053139,...,0.000000,-0.054602,0.017914,-0.047580,-0.051430,-0.023283,0.174763,0.089021,-0.024576,1
2021-02-08,0.000000,0.000000,0.071374,-0.045171,0.000000,-0.012313,-0.041916,0.078294,-0.035286,0.053139,...,0.000000,-0.054602,0.017914,-0.047580,-0.051430,-0.023283,0.174763,0.089021,-0.024576,1
2021-02-10,0.000000,0.000000,0.030380,0.017489,0.000000,0.005509,0.022061,-0.000492,0.009954,0.031103,...,0.000000,-0.039151,0.008010,0.033947,-0.058943,0.087335,0.075167,0.084870,0.032516,1
2021-03-01,0.000000,0.000000,-0.001941,-0.020123,0.000000,-0.036040,-0.027074,-0.005707,0.007087,-0.002945,...,-0.029627,-0.077908,0.052371,0.028151,-0.096228,-0.061104,-0.068241,-0.152699,-0.051254,1
2021-03-04,0.000000,0.000000,0.255754,-0.071696,0.000000,0.041480,0.058806,0.010639,0.024888,-0.005755,...,0.115919,0.018717,0.014414,0.021569,-0.031765,0.029723,0.086343,0.341999,0.033715,1
2021-03-17,0.000000,0.000000,0.196822,0.118644,0.000000,0.010508,-0.000993,-0.039405,0.024902,0.198186,...,0.062740,0.159021,0.004716,0.020055,0.093177,-0.054962,-0.063587,-0.039796,-0.088220,1
2021-03-17,0.000000,0.000000,0.196822,0.118644,0.000000,0.010508,-0.000993,-0.039405,0.024902,0.198186,...,0.062740,0.159021,0.004716,0.020055,0.093177,-0.054962,-0.063587,-0.039796,-0.088220,1


In [38]:
combined_new=combined.drop(['Institutional Adopt'], axis=1)
combined_new

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
2021-01-05,0.000000,0.000000,-0.088201,-0.122700,0.000000,-0.045453,0.037039,0.120509,-0.108976,0.046063,...,-0.063158,0.000000,-0.098702,0.142612,-0.026812,-0.520328,0.059270,-0.080577,0.018532,0.080724
2021-01-28,0.000000,0.000000,-0.036659,0.178912,0.000000,-0.065799,-0.087956,-0.084846,-0.098515,-0.098414,...,0.090114,0.000000,-0.056205,0.037974,-0.093725,-0.182521,-0.139449,-0.112352,-0.104540,-0.068550
2021-01-28,0.000000,0.000000,-0.036659,0.178912,0.000000,-0.065799,-0.087956,-0.084846,-0.098515,-0.098414,...,0.090114,0.000000,-0.056205,0.037974,-0.093725,-0.182521,-0.139449,-0.112352,-0.104540,-0.068550
2021-02-08,0.000000,0.000000,0.071374,-0.045171,0.000000,-0.012313,-0.041916,0.078294,-0.035286,0.053139,...,-0.040114,0.000000,-0.054602,0.017914,-0.047580,-0.051430,-0.023283,0.174763,0.089021,-0.024576
2021-02-08,0.000000,0.000000,0.071374,-0.045171,0.000000,-0.012313,-0.041916,0.078294,-0.035286,0.053139,...,-0.040114,0.000000,-0.054602,0.017914,-0.047580,-0.051430,-0.023283,0.174763,0.089021,-0.024576
2021-02-10,0.000000,0.000000,0.030380,0.017489,0.000000,0.005509,0.022061,-0.000492,0.009954,0.031103,...,0.018177,0.000000,-0.039151,0.008010,0.033947,-0.058943,0.087335,0.075167,0.084870,0.032516
2021-03-01,0.000000,0.000000,-0.001941,-0.020123,0.000000,-0.036040,-0.027074,-0.005707,0.007087,-0.002945,...,-0.049854,-0.029627,-0.077908,0.052371,0.028151,-0.096228,-0.061104,-0.068241,-0.152699,-0.051254
2021-03-04,0.000000,0.000000,0.255754,-0.071696,0.000000,0.041480,0.058806,0.010639,0.024888,-0.005755,...,-0.000634,0.115919,0.018717,0.014414,0.021569,-0.031765,0.029723,0.086343,0.341999,0.033715
2021-03-17,0.000000,0.000000,0.196822,0.118644,0.000000,0.010508,-0.000993,-0.039405,0.024902,0.198186,...,0.001373,0.062740,0.159021,0.004716,0.020055,0.093177,-0.054962,-0.063587,-0.039796,-0.088220
2021-03-17,0.000000,0.000000,0.196822,0.118644,0.000000,0.010508,-0.000993,-0.039405,0.024902,0.198186,...,0.001373,0.062740,0.159021,0.004716,0.020055,0.093177,-0.054962,-0.063587,-0.039796,-0.088220


In [39]:
variation = combined_new
variation.shape

(25, 52)

In [64]:
column_names= list(combined_new)

In [41]:
# calculate mean and variance for each token
annual_returns = []
variance = []
for x in column_names:
    annual_returns.append(combined_new[x].mean()*252)
    variance.append(combined_new[x].std()*np.sqrt(252))
    
    

In [42]:
stats=pd.DataFrame([annual_returns,variance])

In [43]:
stats_df=stats.T

In [44]:
stats_df.columns=('annual_return','variance')

In [45]:
stats_df.index=column_names

In [46]:
stats_df.head()

,annual_return,variance
spell-token,39.805005,11.917818
illuvium,0.706303,1.013636
polyswarm,10.613691,2.601856
hackenai,1.702293,1.451243
mercurial,-3.786581,1.206435


In [47]:
# Create a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [48]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(stats_df)
    inertia.append(k_model.inertia_)

In [49]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,2635.101536
1,2,1064.530580
2,3,375.962039
3,4,175.422859
4,5,93.053686


In [50]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [51]:
# Create and initialize the K-means model instance for 4 clusters
model = KMeans(n_clusters=4, random_state=1)

In [52]:
# Fit the data to the instance of the model
model.fit(stats_df)

# Make predictions about the data clusters using the trained model
crypto_clusters = model.predict(stats_df)
crypto_clusters

array([1, 3, 2, 3, 0, 0, 0, 3, 0, 3, 2, 2, 3, 0, 0, 3, 2, 2, 3, 0, 0, 3,
       0, 0, 3, 3, 2, 3, 0, 2, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 3, 0, 0, 3,
       0, 3, 0, 0, 0, 3, 3, 0], dtype=int32)

In [53]:
len(crypto_clusters)

52

In [54]:
crypto_clusters_df = pd.DataFrame(crypto_clusters)
crypto_clusters_df.head()

,0
0,1
1,3
2,2
3,3
4,0


In [55]:
stats_df['crypto_cluster']=crypto_clusters
stats_df['names']=column_names
stats_df


,annual_return,variance,crypto_cluster,names
spell-token,39.805005,11.917818,1,spell-token
illuvium,0.706303,1.013636,3,illuvium
polyswarm,10.613691,2.601856,2,polyswarm
hackenai,1.702293,1.451243,3,hackenai
mercurial,-3.786581,1.206435,0,mercurial
bitcoin,-1.910961,0.573970,0,bitcoin
ethereum,-1.173550,0.735166,0,ethereum
solana,3.669167,1.361324,3,solana
polkadot,-3.041666,0.994240,0,polkadot
cardano,4.487687,1.208479,3,cardano


In [56]:
# Plot the crypto clusters Data Frame
stats_df.hvplot.scatter(x="annual_return", y="variance", by="crypto_cluster",text='names',text_baseline='top',hover=True, hover_cols=['index'])
#stats_df.hvplot.labels(x='annual_return', y='variance',by='crypto_cluster', text='names',text_baseline='top', hover=True)

:NdOverlay   [crypto_cluster]
   :Scatter   [annual_return]   (variance,index)

In [62]:
len(stats_df)

52

In [57]:
correlations_df=combined_new.corr()
btc_correlations=correlations_df.loc['bitcoin']

In [58]:
btc_correlations=btc_correlations.sort_values()

btc_correlations.hvplot.scatter()

:Scatter   [index]   (bitcoin)

In [59]:
plt.figure(figsize = (16,10))
ax = sns.heatmap(correlations_df, annot=True, linewidths=.5)


In [60]:
correlations_df

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
spell-token,1.000000,0.051155,0.063419,0.007597,-0.051285,-0.177959,0.126363,0.024697,0.035611,0.059124,...,0.015851,-0.071356,0.049579,-0.071331,0.145532,0.051260,-0.071655,-0.094248,-0.053400,0.007405
illuvium,0.051155,1.000000,0.180894,0.354731,0.544666,0.379188,0.501401,0.339649,0.444818,0.310982,...,0.549354,0.548780,0.286432,0.420900,0.379869,0.245704,0.402157,0.013552,0.090347,0.384838
polyswarm,0.063419,0.180894,1.000000,0.192984,0.143980,0.354621,0.179326,-0.085066,0.116834,0.236626,...,0.120069,0.448811,0.431331,-0.128938,0.232167,0.146584,-0.021705,-0.174726,0.175529,-0.118791
hackenai,0.007597,0.354731,0.192984,1.000000,0.137448,0.021963,-0.061472,-0.318781,0.022076,0.245658,...,0.664210,0.224664,0.318598,0.068068,-0.070061,0.280151,-0.281746,-0.348863,-0.250329,-0.247949
mercurial,-0.051285,0.544666,0.143980,0.137448,1.000000,0.371379,0.361192,0.663633,0.645832,0.340378,...,0.609549,0.640764,0.429530,0.577564,0.002433,0.071909,0.630971,0.463780,0.352461,0.652133
bitcoin,-0.177959,0.379188,0.354621,0.021963,0.371379,1.000000,0.776344,0.208583,0.762429,0.523589,...,0.111446,0.572174,0.506771,0.089431,0.506614,0.499485,0.521497,0.279648,0.411137,0.348460
ethereum,0.126363,0.501401,0.179326,-0.061472,0.361192,0.776344,1.000000,0.378720,0.662854,0.521490,...,0.065546,0.565056,0.379940,0.298378,0.651559,0.311604,0.665576,0.117836,0.438428,0.556754
solana,0.024697,0.339649,-0.085066,-0.318781,0.663633,0.208583,0.378720,1.000000,0.522828,0.220463,...,0.158473,0.277593,0.169744,0.476699,0.123239,0.002546,0.851454,0.637666,0.525827,0.892674
polkadot,0.035611,0.444818,0.116834,0.022076,0.645832,0.762429,0.662854,0.522828,1.000000,0.506244,...,0.311442,0.514337,0.608667,0.235296,0.519820,0.549731,0.696972,0.518260,0.398406,0.574623
cardano,0.059124,0.310982,0.236626,0.245658,0.340378,0.523589,0.521490,0.220463,0.506244,1.000000,...,0.057552,0.417018,0.657197,0.217802,0.438143,0.375271,0.291785,0.102507,0.171872,0.075534


In [61]:
# Plot the DataFrame
# Visualization
plt.figure(1, facecolor='w', figsize=(12, 9))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')


#Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)


# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.nipy_spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)

# a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)


# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):
    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.nipy_spectral(label / float(n_labels)),
                       alpha=.6))
plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())
plt.show()

NameError: name 'edge_model' is not defined